In [1]:
import angr, claripy , os


In [2]:
import logging
logging.getLogger('angr').setLevel(logging.INFO)


In [3]:
filePath = '/home/nahid/reverse/binaries/pass'
proj = angr.Project(filePath, auto_load_libs=False)

INFO     | 2023-02-07 13:57:41,952 | angr.project   | Loading binary /home/nahid/reverse/binaries/pass
WARNING  | 2023-02-07 13:57:42,046 | cle.loader     | The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.
INFO     | 2023-02-07 13:57:42,048 | angr.project   | Using builtin SimProcedure for unresolved __libc_start_main from libc.so.0
INFO     | 2023-02-07 13:57:42,049 | angr.project   | Using builtin SimProcedure for unresolved puts from libc.so.0
INFO     | 2023-02-07 13:57:42,050 | angr.project   | Using builtin SimProcedure for unresolved __stack_chk_fail from libc.so.0
INFO     | 2023-02-07 13:57:42,050 | angr.project   | Using builtin SimProcedure for unresolved printf from libc.so.0
INFO     | 2023-02-07 13:57:42,051 | angr.project   | Using builtin SimProcedure for unresolved read from libc.so.0
INFO     | 2023-02-07 13:57:42,052 | angr.project   | Using builtin SimProcedure for unresolved strcmp from libc.so.0
INFO     | 2

In [4]:
entry_state = proj.factory.entry_state()


In [5]:
entry_state

<SimState @ 0x401120>

In [6]:
entry_state.addr

4198688

In [6]:
entry_block = entry_state.block()

In [7]:
entry_block.pp()


        _start:
401120  endbr64 
401124  xor     ebp, ebp
401126  mov     r9, rdx
401129  pop     rsi
40112a  mov     rdx, rsp
40112d  and     rsp, 0xfffffffffffffff0
401131  push    rax
401132  push    rsp
401133  xor     r8d, r8d
401136  xor     ecx, ecx
401138  lea     rdi, [main]
40113f  call    qword ptr [0x403fd8]


In [8]:
proj.factory.simulation_manager?

Signature:
proj.factory.simulation_manager(
    thing: Union[List[angr.sim_state.SimState], angr.sim_state.SimState, NoneType] = None,
    **kwargs,
) -> 'SimulationManager'
Docstring:
Constructs a new simulation manager.

:param thing:           Optional - What to put in the new SimulationManager's active stash (either a SimState or a list of SimStates).
:param kwargs:          Any additional keyword arguments will be passed to the SimulationManager constructor
:returns:               The new SimulationManager
:rtype:                 angr.sim_manager.SimulationManager

Many different types can be passed to this method:

* If nothing is passed in, the SimulationManager is seeded with a state initialized for the program
  entry point, i.e. :meth:`entry_state()`.
* If a :class:`SimState` is passed in, the SimulationManager is seeded with that state.
* If a list is passed in, the list must contain only SimStates and the whole list will be used to seed the SimulationManager.
File:      ~/a

In [10]:
sim_mgr = proj.factory.simulation_manager(entry_state)

In [11]:
sim_mgr.step()

INFO     | 2023-02-07 14:01:25,049 | angr.sim_manager | Stepping active of <SimulationManager with 1 active>
INFO     | 2023-02-07 14:01:25,070 | angr.engines.engine | Ticked state: <IRSB from 0x401120: 1 sat>


<SimulationManager with 1 active>

In [12]:
sim_mgr.active[0].block().pp()


        __libc_start_main:
500000  add     byte ptr [rax], al
500002  add     byte ptr [rax], al
500004  add     byte ptr [rax], al
500006  add     byte ptr [rax], al


In [13]:
sim_mgr.step()

INFO     | 2023-02-07 14:01:31,253 | angr.sim_manager | Stepping active of <SimulationManager with 1 active>
INFO     | 2023-02-07 14:01:31,455 | angr.engines.engine | Ticked state: <SimProcedure __libc_start_main from 0x500000: 1 sat>


<SimulationManager with 1 active>

In [14]:
sim_mgr.active[0].block().pp()

        main:
4012f9  endbr64 
4012fd  push    rbp
4012fe  mov     rbp, rsp
401301  sub     rsp, 0x30
401305  mov     dword ptr [rbp-0x24], edi
401308  mov     qword ptr [rbp-0x30], rsi
40130c  mov     rax, qword ptr  fs:[0x28]
401315  mov     qword ptr [rbp-0x8], rax
401319  xor     eax, eax
40131b  mov     byte ptr [rbp-0x12], 0x0
40131f  mov     byte ptr [rbp-0x9], 0x0
401323  lea     rax, [0x40204d]
40132a  mov     rdi, rax
40132d  call    puts


In [23]:
sim_mgr.step()

INFO     | 2023-02-06 15:47:11,007 | angr.sim_manager | Stepping active of <SimulationManager with 1 active>
INFO     | 2023-02-06 15:47:11,021 | angr.engines.engine | Ticked state: <IRSB from 0x4012f9: 1 sat>


<SimulationManager with 1 active>

In [24]:
sim_mgr.active[0].block().pp()

        puts:
4010b0  endbr64 
4010b4  bnd jmp qword ptr [0x403fa0]
